In [5]:
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.models import load_model
import os


In [6]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [7]:
batch_size = 32
epochs = 10

In [10]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('/Users/pratimakandel/Downloads/data/train/', class_mode='binary', batch_size=64, target_size=(img_width, img_height))
# load and iterate validation dataset
val_it = datagen.flow_from_directory('/Users/pratimakandel/Downloads/data/validation/', class_mode='binary', batch_size=64, target_size=(img_width, img_height))
# load and iterate test dataset
test_it = datagen.flow_from_directory('/Users/pratimakandel/Downloads/data/test/', class_mode='binary', batch_size=64)

Found 803 images belonging to 2 classes.
Found 91 images belonging to 2 classes.
Found 54 images belonging to 2 classes.


In [21]:
img_width, img_height = 150, 150
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [22]:
model = Sequential()
# Padding = 'same'  results in padding the input such that
# the output has the same length as the original input
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape= input_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('softmax'))

In [23]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's create our model
model.compile(loss = 'binary_crossentropy',
              optimizer = opt,
              metrics = ['accuracy'])

In [24]:
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
activation_7 (Activation)    (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 148, 148, 32)      9248      
_________________________________________________________________
activation_8 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 74, 74, 64)       

In [25]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('/Users/pratimakandel/Downloads/data/train/', class_mode='binary', batch_size=64, target_size=(img_width, img_height))
# load and iterate validation dataset
val_it = datagen.flow_from_directory('/Users/pratimakandel/Downloads/data/validation/', class_mode='binary', batch_size=64, target_size=(img_width, img_height))
# load and iterate test dataset
test_it = datagen.flow_from_directory('/Users/pratimakandel/Downloads/data/test/', class_mode='binary', batch_size=64)

Found 803 images belonging to 2 classes.
Found 91 images belonging to 2 classes.
Found 54 images belonging to 2 classes.


In [32]:

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '/Users/pratimakandel/Downloads/data/train/',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '/Users/pratimakandel/Downloads/data/validation/',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')



Found 803 images belonging to 2 classes.
Found 91 images belonging to 2 classes.


In [33]:
model.fit_generator(
        train_generator,
        steps_per_epoch= 2,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=50)
model.save('/Users/pratimakandel/Downloads/first_try.h5')  # always save your weights after training or during training

Epoch 1/1
2/2 [==============================] - 26s 13s/step - loss: 7.9062 - accuracy: 0.4844 - val_loss: 10.9604 - val_accuracy: 0.2632


In [35]:
import cv2
import numpy as np
from keras.models import load_model

img_row, img_height, img_depth = 150,150,3
classifier = load_model('/Users/pratimakandel/Downloads/first_try.h5')
img_path = '/Users/pratimakandel/Downloads/data/validation/metal/metal62.jpg'
color = True 
scale = 8

def draw_test(name, res, input_im, scale, img_row, img_height):
    BLACK = [0,0,0]
    res = int(res)
    if res == 0:
        pred = "plastic"
    if res == 1:
        pred = "metal"
    
        
    expanded_image = cv2.copyMakeBorder(input_im, 0, 0, 0, imageL.shape[0]*2 ,cv2.BORDER_CONSTANT,value=BLACK)
    if color == False:
        expanded_image = cv2.cvtColor(expanded_image, cv2.COLOR_GRAY2BGR)
    cv2.putText(expanded_image, str(pred), (300, 80) , cv2.FONT_HERSHEY_COMPLEX_SMALL,3, (0,255,0), 2)
    cv2.imshow(name, expanded_image)


for i in range(0,10):
    rand = np.random.randint(0,len(validation_generator))
    input_im = validation_generator[rand]
    imageL = cv2.resize(np.float32(input_im), None, fx=scale, fy=scale, interpolation = cv2.INTER_CUBIC) 
    input_im = input_im.reshape(1,img_row, img_height, img_depth) 
    
    ## Get Prediction
    res = str(classifier.predict_classes(input_im, 1, verbose = 0)[0])
              
    draw_test("Prediction", res, imageL, scale, img_row, img_height) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

ValueError: could not broadcast input array from shape (32,150,150,3) into shape (32)